Step 1: Install and Imports

In [2]:
!pip install --quiet google-cloud-bigquery google-cloud-storage requests pandas

In [3]:
import os
import json
import time
import requests
import pandas as pd
from datetime import datetime, timezone

from google.cloud import bigquery
from google.cloud import storage


Step 2: Variables Declaration

In [7]:
PROJECT_ID = "qwiklabs-gcp-00-e063736770a2"

DATASET_ID = "aero_alerts"
RAW_TABLE = "airports_raw"
AIRPORTS_TABLE = "airports_large"
FORECAST_TABLE = "airport_forecasts"
ALERTS_TABLE = "airport_alerts"

BUCKET_NAME = f"{PROJECT_ID}-aero-alerts"  # if creation fails, we will switch to an existing bucket
GCS_OBJECT = "input/airports.csv"

SOURCE_GCS_URI = "gs://labs.roitraining.com/data-to-ai-workshop/airports.csv"


Step 3: Create Dataset

In [8]:
bq = bigquery.Client(project=PROJECT_ID)

dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"

bq.create_dataset(dataset, exists_ok=True)
print("Dataset created.")


Dataset created.


Step 4: Create our own GCS bucket to copy CSV into it for reliability and reuse

---



4.1 Create bucket

In [9]:
gcs = storage.Client(project=PROJECT_ID)

try:
    bucket = gcs.bucket(BUCKET_NAME)
    bucket.location = "US"
    gcs.create_bucket(bucket)
    print(f"Bucket created: {BUCKET_NAME}")
except Exception as e:
    print(f"Bucket create skipped or failed: {e}")


/tmp/ipython-input-150221518.py:5: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = "US"


Bucket created: qwiklabs-gcp-00-e063736770a2-aero-alerts


4.2 Copy from training bucket into our bucket

In [10]:
src_bucket_name = "labs.roitraining.com"
src_object = "data-to-ai-workshop/airports.csv"

src_bucket = gcs.bucket(src_bucket_name)
src_blob = src_bucket.blob(src_object)

dst_bucket = gcs.bucket(BUCKET_NAME)
dst_blob = dst_bucket.blob(GCS_OBJECT)

dst_blob.rewrite(src_blob)
print(f"Copied to gs://{BUCKET_NAME}/{GCS_OBJECT}")


Copied to gs://qwiklabs-gcp-00-e063736770a2-aero-alerts/input/airports.csv


Step 5: Load CSV from GCS into BigQuery Raw Table

In [11]:
raw_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}"
gcs_uri = f"gs://{BUCKET_NAME}/{GCS_OBJECT}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE",
)

load_job = bq.load_table_from_uri(gcs_uri, raw_table_ref, job_config=job_config)
load_job.result()

print("Loaded airports_raw.")


Loaded airports_raw.


In [18]:
sql = f"""
select *
from `{raw_table_ref}`
limit 10
"""

bq.query(sql).to_dataframe()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11,NA,US,US-PA,Bensalem,False,None,None,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,None,None
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435,NA,US,US-KS,Leoti,False,None,None,00AA,00AA,None,None,None
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450,NA,US,US-AK,Anchor Point,False,None,None,00AK,00AK,None,None,None
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820,NA,US,US-AL,Harvest,False,None,None,00AL,00AL,None,None,None
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80,NA,US,US-AK,King Salmon,False,None,None,00AN,00AN,None,None,None
5,322127,00AS,small_airport,Fulton Airport,34.942803,-97.818019,1100,NA,US,US-OK,Alex,False,None,None,00AS,00AS,None,None,None
6,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810,NA,US,US-AZ,Cordes,False,None,None,00AZ,00AZ,None,None,None
7,6528,00CA,small_airport,Goldstone (GTS) Airport,35.354740,-116.885329,3038,NA,US,US-CA,Barstow,False,None,None,00CA,00CA,None,https://en.wikipedia.org/wiki/Goldstone_Gts_Ai...,None
8,324424,00CL,small_airport,Williams Ag Airport,39.427188,-121.763427,87,NA,US,US-CA,Biggs,False,None,None,00CL,00CL,None,None,None
9,322658,00CN,heliport,Kitchen Creek Helibase Heliport,32.727374,-116.459742,3350,NA,US,US-CA,Pine Valley,False,None,None,00CN,00CN,None,None,None


Step 6: Create a Clean Large-Airports Table

6.1 Inspect schema

In [12]:
table = bq.get_table(raw_table_ref)
[col.name for col in table.schema]

['id',
 'ident',
 'type',
 'name',
 'latitude_deg',
 'longitude_deg',
 'elevation_ft',
 'continent',
 'iso_country',
 'iso_region',
 'municipality',
 'scheduled_service',
 'icao_code',
 'iata_code',
 'gps_code',
 'local_code',
 'home_link',
 'wikipedia_link',
 'keywords']

6.2 Create large-airport subset for US

In [19]:
airports_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{AIRPORTS_TABLE}"

sql = f"""
create or replace table `{airports_table_ref}` AS
select
  coalesce(iata_code, ident, CAST(id AS STRING)) AS airport_id,
  name AS airport_name,
  type AS airport_type,
  latitude_deg AS lat,
  longitude_deg AS lon,
  municipality,
  iso_region,
  iso_country
from `{raw_table_ref}`
where lower(type) = 'large_airport'
  and latitude_deg is not null
  and longitude_deg is not null
  and iso_country = 'US'
"""
bq.query(sql).result()
print("Created airports_large.")


Created airports_large.


Step 7: Pull Forecasts from National Weather Service API for Each Large Airport

In [20]:
def nws_get_json(url: str) -> dict:
    headers = {
        "User-Agent": "AeroAlertsWorkshop/1.0 (contact: student@example.com)",
        "Accept": "application/geo+json, application/json",
    }
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    return r.json()


def build_forecast_summary(forecast_json: dict, max_periods: int = 6) -> str:
    periods = (forecast_json.get("properties", {}) or {}).get("periods", []) or []
    periods = periods[:max_periods]

    lines = []
    for p in periods:
        name = p.get("name")
        temp = p.get("temperature")
        temp_unit = p.get("temperatureUnit")
        wind = p.get("windSpeed")
        wind_dir = p.get("windDirection")
        short = p.get("shortForecast")
        detailed = p.get("detailedForecast")
        lines.append(
            f"{name}: {short}. Temp {temp}{temp_unit}. Wind {wind} {wind_dir}. Details: {detailed}"
        )
    return "\n".join(lines)


Pull and stage forecasts into a DataFrame

In [21]:
airports_df = bq.query(f"SELECT airport_id, airport_name, lat, lon FROM `{airports_table_ref}`").to_dataframe()
print(f"Large airports: {len(airports_df)}")
airports_df.head()


Large airports: 71


,airport_id,airport_name,lat,lon
0,ANC,Ted Stevens Anchorage International Airport,61.179004,-149.992561
1,PHX,Phoenix Sky Harbor International Airport,33.435302,-112.005905
2,SNA,John Wayne Orange County International Airport,33.675701,-117.867996
3,ONT,Ontario International Airport,34.056000,-117.600998
4,LAX,Los Angeles International Airport,33.942501,-118.407997


In [22]:
rows = []
run_ts = datetime.now(timezone.utc).isoformat()

for idx, row in airports_df.iterrows():
    airport_id = row["airport_id"]
    airport_name = row["airport_name"]
    lat = row["lat"]
    lon = row["lon"]

    try:
        points_url = f"https://api.weather.gov/points/{lat},{lon}"
        points_json = nws_get_json(points_url)

        forecast_url = points_json["properties"]["forecast"]
        forecast_json = nws_get_json(forecast_url)

        summary = build_forecast_summary(forecast_json, max_periods=6)

        rows.append({
            "run_ts_utc": run_ts,
            "airport_id": airport_id,
            "airport_name": airport_name,
            "lat": float(lat),
            "lon": float(lon),
            "forecast_url": forecast_url,
            "forecast_summary": summary
        })

        time.sleep(0.2)  # gentle pacing
    except Exception as e:
        rows.append({
            "run_ts_utc": run_ts,
            "airport_id": airport_id,
            "airport_name": airport_name,
            "lat": float(lat),
            "lon": float(lon),
            "forecast_url": None,
            "forecast_summary": f"ERROR: {str(e)}"
        })

forecast_df = pd.DataFrame(rows)
forecast_df.head()


,run_ts_utc,airport_id,airport_name,lat,lon,forecast_url,forecast_summary
0,2026-01-16T19:14:44.585320+00:00,ANC,Ted Stevens Anchorage International Airport,61.179004,-149.992561,"https://api.weather.gov/gridpoints/AER/141,235...",Today: Rain And Snow then Chance Snow Showers....
1,2026-01-16T19:14:44.585320+00:00,PHX,Phoenix Sky Harbor International Airport,33.435302,-112.005905,"https://api.weather.gov/gridpoints/PSR/161,57/...",This Afternoon: Sunny. Temp 77F. Wind 0 to 5 m...
2,2026-01-16T19:14:44.585320+00:00,SNA,John Wayne Orange County International Airport,33.675701,-117.867996,"https://api.weather.gov/gridpoints/SGX/38,60/f...",Today: Sunny. Temp 78F. Wind 5 mph NW. Details...
3,2026-01-16T19:14:44.585320+00:00,ONT,Ontario International Airport,34.056000,-117.600998,"https://api.weather.gov/gridpoints/SGX/50,75/f...",Today: Sunny. Temp 81F. Wind 5 to 10 mph NW. D...
4,2026-01-16T19:14:44.585320+00:00,LAX,Los Angeles International Airport,33.942501,-118.407997,"https://api.weather.gov/gridpoints/LOX/148,41/...",Today: Sunny. Temp 75F. Wind 5 mph WSW. Detail...


Step 8: Write Forecasts to BigQuery

In [26]:
forecast_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{FORECAST_TABLE}"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
    autodetect=True
)

bq.load_table_from_dataframe(forecast_df, forecast_table_ref, job_config=job_config).result()
print("Loaded airport_forecasts.")


Loaded airport_forecasts.


Step 9: Create Gemini Remote Model in BigQuery

In [27]:
MODEL_NAME = "gemini_model"
model_ref = f"{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}"

sql = f"""
CREATE OR REPLACE MODEL `{model_ref}`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'gemini-2.0-flash-001');
"""
bq.query(sql).result()
print("Gemini remote model created.")


Gemini remote model created.


Step 10: Generate Alerts with ML.GENERATE_TEXT and Store in BigQuery

In [28]:
alerts_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{ALERTS_TABLE}"

sql = f"""
create or replace table `{alerts_table_ref}` as
select
     run_ts_utc
    ,airport_id
    ,airport_name
    ,lat
    ,lon
    ,forecast_url
    ,forecast_summary
    ,json_value(
        ml_generate_text_result,
        '$.candidates[0].content.parts[0].text'
     ) as alert_text
from ML.GENERATE_TEXT(
    model `{model_ref}`,
    (
        select
             run_ts_utc
            ,airport_id
            ,airport_name
            ,lat
            ,lon
            ,forecast_url
            ,forecast_summary
            ,format(
                '''Create a concise operational weather alert for an airport.
Airport: %s (%s)
Write 3 lines:
1) Risk level: Low, Medium, High
2) Key hazards in plain language
3) Recommended action for airport ops

Forecast:
%s''',
                airport_name,
                airport_id,
                forecast_summary
            ) as prompt
        from `{forecast_table_ref}`
        where not starts_with(forecast_summary, 'ERROR:')
    ),
    struct(
         0.2 as temperature
        ,512 as max_output_tokens
    )
)
"""
bq.query(sql).result()
print("Created airport_alerts.")


Created airport_alerts.


Validate the alerts table

In [29]:
sql = f"""
select
     airport_id
    ,airport_name
    ,alert_text
from `{alerts_table_ref}`
limit 10
"""

bq.query(sql).to_dataframe()

,airport_id,airport_name,alert_text
0,JAX,Jacksonville International Airport,Here's a concise operational weather alert for...
1,BOS,Logan International Airport,Here's a concise operational weather alert for...
2,MIA,Miami International Airport,Here's a concise operational weather alert for...
3,PVD,Theodore Francis Green State Airport,**Airport Weather Alert - PVD**\n\n1. Risk Le...
4,MSY,Louis Armstrong New Orleans International Airport,**MSY Operational Weather Alert:**\n\n1. Risk...
5,OAK,San Francisco Bay Oakland International Airport,**OAK Operational Weather Alert:**\n\n1. **Ri...
6,MDW,Chicago Midway International Airport,Here's a concise operational weather alert for...
7,SRQ,Sarasota Bradenton International Airport,Here's a concise operational weather alert for...
8,BNA,Nashville International Airport,Here's a concise operational weather alert for...
9,OGG,Kahului International Airport,Here's a concise operational weather alert for...


**Automation**

This notebook is scheduled to run daily using Colab Enterprise scheduling to refresh weather data and regenerate Gemini-based airport alerts in BigQuery.